# Preparation

In [1]:
import numpy as np
import time
from annoy import AnnoyIndex
import faiss
import hnswlib

# 1. Generate Dataset

In [2]:
# ===============================
n_data = 1_000_000   # bisa coba 100_000 dulu jika RAM terbatas
dim = 5
X = np.random.random((n_data, dim)).astype(np.float32)

# Query point
query = np.random.random((1, dim)).astype(np.float32)
k = 10

# 2. ANNOY

In [3]:
print("=== Annoy ===")
ann_index = AnnoyIndex(dim, 'euclidean')

start = time.time()
for i in range(n_data):
    ann_index.add_item(i, X[i])
ann_index.build(10)  # 10 trees
build_time = time.time() - start

start = time.time()
neighbors = ann_index.get_nns_by_vector(query[0], k, include_distances=True)
query_time = time.time() - start

print("Build time:", build_time, "detik")
print("Query time:", query_time, "detik")
print("Neighbors:", neighbors[0][:5], "...")

=== Annoy ===
Build time: 5.0234246253967285 detik
Query time: 0.0005936622619628906 detik
Neighbors: [29848, 804607, 333980, 76813, 36420] ...


# 3. FAISS

In [4]:
print("\n=== FAISS (IndexFlatL2) ===")
faiss_index = faiss.IndexFlatL2(dim)

start = time.time()
faiss_index.add(X)
build_time = time.time() - start

start = time.time()
distances, indices = faiss_index.search(query, k)
query_time = time.time() - start

print("Build time:", build_time, "detik")
print("Query time:", query_time, "detik")
print("Neighbors:", indices[0][:5], "...")


=== FAISS (IndexFlatL2) ===
Build time: 0.01844930648803711 detik
Query time: 0.005891323089599609 detik
Neighbors: [ 29848 804607 333980  76813  36420] ...


# 4. HNSW

In [5]:
print("\n=== HNSW (hnswlib) ===")
hnsw_index = hnswlib.Index(space='l2', dim=dim)

start = time.time()
hnsw_index.init_index(max_elements=n_data, ef_construction=200, M=16)
hnsw_index.add_items(X)
build_time = time.time() - start

hnsw_index.set_ef(50)

start = time.time()
labels, distances = hnsw_index.knn_query(query, k=k)
query_time = time.time() - start

print("Build time:", build_time, "detik")
print("Query time:", query_time, "detik")
print("Neighbors:", labels[0][:5], "...")


=== HNSW (hnswlib) ===
Build time: 61.93322944641113 detik
Query time: 0.0048923492431640625 detik
Neighbors: [ 29848 804607 333980  76813  36420] ...
